In [ ]:
import os
from pprint import pprint

GT_DIR = "./oxford5k_gt"
IMAGE_DIR = "./oxford5k_images"
OUTPUT_QUERY_IMAGE_DIR = "./oxford5k_query_images"
if not os.path.exists(OUTPUT_QUERY_IMAGE_DIR):
    os.mkdir(OUTPUT_QUERY_IMAGE_DIR)
OUTPUT_QUERY_IMAGE_LIST_PATH = "./oxford5k_query_image_list.txt"

files = os.listdir(GT_DIR)
filename_set = set()
for filename in files:
    filename = filename.replace(".txt", "")\
                        .replace("_ok", "") \
                        .replace("_good", "") \
                        .replace("_junk", "") \
                        .replace("_query", "")
                     
    # print(filename)
    filename_set.add(filename)
    
query_names = list(filename_set)
query_names.sort()

print("{} queres exists".format(len(query_names)))
pprint(query_names)

# Generate query image path list from Oxford5k queries

This script generate `image path list` file which is used in colmap's image retrieval system. 
You have to feed it to the image retrieval system to get `rank list` files for each query. 

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

with open(OUTPUT_QUERY_IMAGE_LIST_PATH, "w") as list_file:
    for query_name in query_names:
        with open(os.path.join(GT_DIR, query_name+"_query.txt")) as f:
            query_info = f.readline().strip().split(" ")
        # print(query_info)

        # TODO: how to crop with float values? do we need interpolation?
        x_st = int(float(query_info[1]))
        y_st = int(float(query_info[2]))
        x_en = int(float(query_info[3]))
        y_en = int(float(query_info[4]))

        image_name = query_info[0].replace("oxc1_", "")
        image_path = os.path.join(IMAGE_DIR, image_name+".jpg")
        # print("open image:", image_path)
        img_bgr = cv2.imread(image_path)
        crop_img_bgr = img_bgr[y_st:y_en, x_st:x_en]


        img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        crop_img = cv2.cvtColor(crop_img_bgr, cv2.COLOR_BGR2RGB)

        query_filename = query_name+".png"
        query_image_path = os.path.abspath(os.path.join(OUTPUT_QUERY_IMAGE_DIR, query_filename))
        cv2.imwrite(query_image_path, img_bgr)
        # cv2.imwrite(query_image_path, crop_img)
        list_file.write(query_filename+"\n")

    #     plt.figure()
    #     plt.imshow(img)
    #     plt.show()

    #     plt.figure()
    #     plt.imshow(crop_img)
    #     plt.show()
print("done")

# Compute mAP with rank list files

In [ ]:
QUERY_RESULT_PATH = "./query_result.txt"

OUTPUT_RANK_LIST_DIR = "./oxford5k_rank_list"
if not os.path.exists(OUTPUT_RANK_LIST_DIR):
    os.mkdir(OUTPUT_RANK_LIST_DIR)

fh = None
with open(QUERY_RESULT_PATH, "r") as f:
    for line_idx, line in enumerate(f.readlines()):
        # print(line)
        if "Querying for image " in line:
            query_name = line.split(" ")[3]            
            if not fh is None:
                fh.close()
            fh = open(os.path.join(OUTPUT_RANK_LIST_DIR, query_name+"_rank.txt"), "w")
        else:      
            result_image_filename = line.strip().split(", ")[1].split("=")[1]
            result_image_name = result_image_filename.replace(".jpg", "")
            fh.write(result_image_name+"\n")

In [ ]:
import os
import subprocess
from subprocess import CalledProcessError
import shlex

EXE_DIR_PATH = "."
WORK_DIR = "../ir_demo/"

def run_cmd(cmd, cwd=None):
    """
     limitation: cannot see progress until the process is finished. Not suitable for long-running task. 
    """
    try:       
        output = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, cwd=cwd)
        output = output.decode('utf-8')
    except CalledProcessError as e:
        print(e)
        print("output: {}".format(e.output.decode('utf-8')))
    return output
        

cmd = "ls {EXE_DIR_PATH}".format( \
        EXE_DIR_PATH=EXE_DIR_PATH)
output = run_cmd(cmd)
print(output)

In [ ]:
OXFORD_GT_WORKDIR = "./oxford5k_gt/"
# Relative PATH according to above work dir
RANK_LIST_DIR = "./oxford5k_rank_list"
COMPUTE_AP_EXE_PATH = "./oxford5k_compute_ap"

files = os.listdir(RANK_LIST_DIR)
ap_sum = 0.0
for f in files:
    query_name = f.split(".")[0]
    
    cmd = "{COMPUTE_AP_EXE_PATH} {QUERY_NAME} {RANK_LIST_FILE_PATH}".format(\
        COMPUTE_AP_EXE_PATH=COMPUTE_AP_EXE_PATH, \
        QUERY_NAME = query_name, \
        RANK_LIST_FILE_PATH = os.path.abspath(os.path.join(RANK_LIST_DIR, f)))
    output = run_cmd(cmd, cwd=OXFORD_GT_WORKDIR)
    print("cmd: ", cmd)
    print("AP: ", output)
    ap = float(output)
    ap_sum += ap
    
ap_mean = ap_sum / len(files)
print("mAP:", ap_mean)
